# Short-term scheduling

Here we present the short-term scheduling module. The short-term scheduling module of spock has been concived to modify existing plan in order to add special or follow up observations, like transits or monitoring for instance.

<div class="alert alert-info">

**Note:** Before you insert new observations and upload them please make sure you have informed the SPECULOOS consortium 

</div>

## 1- Special target

<div class="alert alert-warning">
    
**WARNING:** First of all, users must ensure that they have entered requiered information of the target they wish to schedule either in <span style="color:teal"> *target_list_special.txt* </span> if it's a special observation or in <span style="color:teal"> *target_list_follow_up.txt* </span> if it's a transiting planet follow up.

</div>


For a given night and telescope here is hiw you get its corresponding **|night blocks|**.

In [1]:
import SPOCK.stats as SPOCKstats

SPOCKstats.read_night_plans_server(telescope='Europa',date='2020-12-12')

INFO:  Please add password.csv file in: /Users/elsaducrot/elsenv/lib/python3.7/site-packages/SPOCK-0.0.1-py3.7.egg/SPOCK/credentials/
INFO:  OK Password file exists


,target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
0,NGTS-11,2020-12-13 00:21:40.021,2020-12-13 05:26:40.021,305.000000,1.0,34.0,5.1480,-14.0,-25.0,-8.93640,"{'texp=10', 'filt=g'}"
1,Sp0535-0931,2020-12-13 05:26:40.021,2020-12-13 08:50:00.020,203.333317,5.0,35.0,21.3006,-9.0,-31.0,-6.52332,"{'filt=I+z', 'texp=25'}"


### For a special observation with given start/end time 

First you want to create the **|night blocks|**. The only esstial information are: 

+ the name of the site, ex: 'SSO'

+ the name of the telescope (because there can be several telescope per site), ex: 'Europa'

+ the date the night starts, ex: '2020-12-12 15:00:00'

+ and the name of your target, 'Trappist-1'

+ start and end time in a list, ex: ['2020-12-12 23:00:00','2020-12-13 01:00:00']

<span style="color:teal">NB: you must first have filled info on the target in *target_list_special.txt* beforehand </span>

This is the block you wish to insert:

In [2]:
import SPOCK.short_term_scheduler as SPOCKST
from astropy.time import Time

schedule = SPOCKST.Schedules()

schedule.load_parameters(obs_name='SSO')
schedule.day_of_night = Time('2020-12-12 15:00:00')
schedule.start_end_range = Time(['2020-12-12 23:00:00','2020-12-13 01:00:00'])
schedule.telescope = 'Europa'
schedule.special_target_with_start_end(input_name="Trappist-1")



target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str10,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
Trappist-1,2020-12-12 23:00:00.018,2020-12-13 01:00:00.018,120.00000000000006,23.0,6.0,29.372136000018543,-5.0,-2.0,-29.031720000000334,"{'texp=50', ""filt=r'""}"


Now you modify existing plan to insert your observation block:

In [3]:
schedule.make_scheduled_table()
schedule.planification()
schedule.make_night_block()

INFO:  Local path does not exist yet 
INFO:  situation 1
INFO:  situation 2
INFO:  situation 1
INFO:  situation 3, no change made to initial schedule
INFO:  no transition block


Here is how the modified **|night blocks|** looks like:

In [4]:
display(schedule.scheduled_table_sorted)

target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str11,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
Trappist-1,2020-12-13 00:21:24.626,2020-12-13 01:00:00.018,120.00000000000006,23.0,6.0,29.372136000018543,-5.0,-2.0,-29.031720000000334,"{'texp=50', ""filt=r'""}"
NGTS-11,2020-12-13 01:00:00.018,2020-12-13 05:26:40.021,266.66671666666673,1.0,34.0,5.148000000001645,-14.0,-25.0,-8.93639999999678,"{'texp=10', 'filt=g'}"
Sp0535-0931,2020-12-13 05:26:40.021,2020-12-13 08:50:00.020,203.33331666666658,5.0,35.0,21.300600000004266,-9.0,-31.0,-6.5233199999974545,"{'filt=I+z', 'texp=25'}"


### For a special target that you wish to observe as much as possible

First you want to create the **|night blocks|**. The only essential information needed are: 

+ the name of the site, ex: 'SSO'

+ the name of the telescope (because there can be several telescope per site), ex: 'Europa'

+ the date the night starts, ex: '2020-12-12 15:00:00'

+ and the name of your target, 'Trappist-1'

<span style="color:teal">NB: you must first have filled info on the target in *target_list_special.txt* beforehand </span>

In [7]:
import SPOCK.short_term_scheduler as SPOCKST
from astropy.time import Time

schedule = SPOCKST.Schedules()

schedule.load_parameters(obs_name= 'SNO')
schedule.day_of_night = Time('2020-12-10 15:00:00')
schedule.telescope = 'Artemis'
schedule.special_target(input_name="NGTS-11")



target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str7,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
NGTS-11,2020-12-10 19:05:40.009,2020-12-11 00:49:00.009,343.3333333333333,1.0,34.0,5.148000000001645,-14.0,-25.0,-8.93639999999678,"{'texp=10', 'filt=g'}"


In [8]:
schedule.make_scheduled_table()
schedule.planification()
schedule.make_night_block()

INFO:  Local path does not exist yet 
INFO:  situation 4
INFO:  situation 2
INFO:  no transition block


Here is how the modified **|night blocks|** looks like:

In [9]:
display(schedule.scheduled_table_sorted)

target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str11,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
NGTS-11,2020-12-10 19:05:40.009,2020-12-11 00:49:00.009,343.3333333333333,1.0,34.0,5.148000000001645,-14.0,-25.0,-8.93639999999678,"{'texp=10', 'filt=g'}"
Sp0840+1824,2020-12-11 00:49:00.009,2020-12-11 06:53:20.009,364.33333333333326,8.0,40.0,29.634552000006664,18.0,24.0,8.726760000003821,"{'texp=26', 'filt=I+z'}"


### Save your plans and night blocks

If you are satisfied with this modified **|night blocks|** you can save it. Executing this cell will upadate your local spock database. The **|night blocks|** will be saved in `your_spock_path + "/DATABASE/telescope/Archive_night_blocks/"`

In [10]:
SPOCKST.save_schedule(input_file=None,nb_observatory=None,save=True,
                      over_write=True,date_range=schedule.day_of_night,telescope=schedule.telescope)


INFO : "/Users/elsaducrot/spock/night_blocks_propositions/night_blocks_Artemis_2020-12-10.txt" has been over-written to "/Users/elsaducrot/spock/DATABASE/Artemis/"
INFO : "/Users/elsaducrot/spock/night_blocks_propositions/night_blocks_Artemis_2020-12-10.txt" has been over-written to "/Users/elsaducrot/spock/DATABASE/Artemis/Archive_night_blocks/"


Now that you have saved the night block it's time to create the ACP plans to send to the control PC of the corresonding telescope. To do so you just have to execute the cell below with the following information:

* `day`, start date you wish to make ACP plans for

* `nb_days`, number of days you wish to make plan for (starting at the date define in `day`)

* `telescope`, name of the telescope at sake

ACP plans will be saved in `your_spock_path + "/DATABASE/telescope/Plans_by_date/"`

In [11]:
SPOCKST.make_plans(day = schedule.day_of_night,nb_days=1,telescope = schedule.telescope)


Now you can upload these ACP plans and **|night blocks|** to the [Cambridge Archive](http://www.mrao.cam.ac.uk/SPECULOOS/) as well as the plans sent to the control PCs. You will just need to provide :

* `day`, start date you for which you wish to upload plans and **|night blocks|**

* `nb_days`, number of days you wish to upload (starting at the date define in `day`)

* `telescope`, name of the telescope at sake

In [12]:
SPOCKST.upload_plans(schedule.day_of_night, nb_days=1,telescope = schedule.telescope)

-----> 2020-12-10 Plans uploaded on the Cambridge server
-----> 2020-12-10 Night plans uploaded on the Cambridge server
-----> 2020-12-10 Zip Plans_by_dates folder uploaded on the Cambridge server
-----> 2020-12-10 Zip Plans_by_dates folder uploaded on the HUB for Artemis
INFO:  Path local 'Gant chart' =  /Users/elsaducrot/spock/SPOCK_Figures/Preview_schedule.html
INFO:  Path database = 'Gant chart' =  speculoos@appcs.ra.phy.cam.ac.uk:/appct/data/SPECULOOSPipeline/Preview_schedule.html


## 2- Follow-up

In [ ]:
import SPOCK.stats as SPOCKstats

SPOCKstats.read_night_plans_server(telescope='Artemis',date='2020-11-16')

In [ ]:
import SPOCK.short_term_scheduler as SPOCKST
from astropy.time import Time

schedule = SPOCKST.Schedules()

schedule.load_parameters(obs_name= 'SNO')
schedule.day_of_night = Time('2020-09-15 15:00:00')
schedule.telescope = 'Artemis'

schedule.transit_follow_up(name="Trappist-1b")

In [ ]:
schedule.make_scheduled_table()
schedule.planification()
schedule.make_night_block()

In [ ]:
display(schedule.scheduled_table_sorted)

To save your night block you should proceed similarly to [the save plans and nights blocks section](###-Save-your-plans-and-night-blocks)

In [ ]:
SPOCKST.save_schedule(input_file=None,nb_observatory=None,save=True,
                      over_write=True,date_range=schedule.day_of_night,telescope=schedule.telescope)
